In [2]:
from wrf import getvar
from netCDF4 import Dataset
import numpy as np

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject


In [3]:
import time

wrfin = Dataset('./wrfout_d02_2023-07-21_18:00:00')

In [7]:
def _extract_ghg(wrffile: Dataset, chem: str):
        if chem == 'xch4':
            _ant = wrffile['CH4_ANT'][0, ...]
            _bck = wrffile['CH4_BCK'][0, ...]
            _bio = wrffile['CH4_BIO'][0, ...]
        elif chem == 'xco2':
            _ant = wrffile['CO2_ANT'][0, ...]
            _bck = wrffile['CO2_BCK'][0, ...]
            _bio = wrffile['CO2_BIO'][0, ...]
        elif chem == 'xco':
            _ant = wrffile['CO_ANT'][0, ...]
            _bck = wrffile['CO_BCK'][0, ...]
            #_bio = wrffile['CO_BIO'][0, ...] ##?
            _bio = np.zeros_like(_bck)
        _ghg = _bio + _ant -_bck
        p = getvar(wrfin, 'P',meta=False)
        sfc_pres = getvar(wrfin, 'PSFC', meta=False)
        if len(_ghg) == len(p):
            pres_bound = np.empty_like(p)
            for i, pres in enumerate(p):
                if i == 0:
                    pres_bound[i] = sfc_pres
                    pres_bound[i+1] = pres_bound[i] + (2*(pres-pres_bound[i]))
                else:
                    try:
                        pres_bound[i+1] = pres_bound[i] + (2*(pres-pres_bound[i]))
                    except IndexError:
                        pass
        p_layer_diff = np.array([pres_bound[i]-pres_bound[i-1] for i in range(1,len(pres_bound))]) #! This may need a value at beginning for xch4[0]
        p_layer_diff = np.insert(p_layer_diff, [0], p[0,...]-sfc_pres, axis=0)
        p_diff = pres_bound[0] - pres_bound[-1]
        return np.sum(_ghg*p_layer_diff)/p_diff

In [8]:
start = time.time()
ghg_python = _extract_ghg(wrfin, 'xch4')
end = time.time()
print(f'Mainly python: {(end-start):0.4f} s')

start = time.time()
ghg_getvar = getvar(wrfin, 'xghg')
end = time.time()
print(f'getvar: {(end-start):0.4f} s')

Mainly python: 0.08050990104675293 s


ValueError: invalid shape for argument 1 (got (47, 255, 300), expected (255, 300))

In [5]:
sfc_p = getvar(wrfin, 'PSFC')

sfc_p

<xarray.DataArray 'PSFC' (south_north: 255, west_east: 300)> Size: 306kB
array([[ 99340.37 ,  99345.36 ,  99349.98 , ..., 100734.42 , 100734.35 ,
        100734.67 ],
       [ 99303.21 ,  99311.266,  99315.445, ..., 100732.34 , 100733.12 ,
        100734.19 ],
       [ 99265.07 ,  99274.15 ,  99278.93 , ..., 100732.07 , 100731.44 ,
        100733.836],
       ...,
       [ 93755.04 ,  93721.805,  93690.1  , ...,  99931.79 ,  99947.99 ,
         99968.06 ],
       [ 93765.41 ,  93733.96 ,  93702.164, ...,  99925.1  ,  99941.24 ,
         99960.58 ],
       [ 93777.36 ,  93743.29 ,  93706.53 , ...,  99912.97 ,  99934.305,
         99954.26 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 306kB -75.46 -75.45 ... -72.07
    XLAT     (south_north, west_east) float32 306kB 40.29 40.29 ... 41.52 41.52
    XTIME    float32 4B 2.52e+03
    Time     datetime64[ns] 8B 2023-07-21T18:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  SFC PRESSURE
    units:        Pa
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-97.0, moad_cen_lat=40.000007629...

In [6]:
from wrf._wrffortran import dcomputexghg
from wrf import to_np, getvar
import numpy as np
from netCDF4 import Dataset
wrfin = Dataset('./wrfout_d02_2023-07-21_18:00:00')
sfc_p = getvar(wrfin, 'PSFC')
p = getvar(wrfin, 'P', meta=False)
ant = getvar(wrfin, 'CH4_ANT', meta=False)
bio = getvar(wrfin, 'CH4_BIO', meta=False)
bck = getvar(wrfin, 'CH4_BCK', meta=False)
out = np.empty_like(to_np(sfc_p))

result = dcomputexghg(np.asfortranarray(to_np(sfc_p)), np.asfortranarray(p), np.asfortranarray(ant), np.asfortranarray(bio), np.asfortranarray(bck), np.asfortranarray(out))

print(result.shape)

ValueError: _wrffortran._wrffortran.dcomputexghg: failed to create array from the 2nd argument `pres` -- 0-th dimension must be fixed to 255 but got 47

In [8]:
p.shape

(47, 255, 300)